<a href="https://colab.research.google.com/github/Raghavnadiminti/Caluclator/blob/main/toxic_comment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [ ]:
non_toxic=data[data['toxic']==0]
threat=data[data['threat']==1]
toxic=data[data['toxic']==1]
severe_toxic=data[data['severe_toxic']==1]
obscene=data[data['obscene']==1]
insult=data[data['insult']==1]
identity_hate=data[data['identity_hate']==1]

In [ ]:
non_toxic = non_toxic.sample(n=16000)


data=pd.concat([non_toxic,toxic], ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,3f252c5b8db86046,Christmas Message \n\nI hope you die in hell y...,1,1,1,1,1,0
1,eb472c7abd1cc98d,subversive activities\nJoseph Cardinal support...,0,0,0,0,0,0
2,774bfac348db66c6,{{unblock|You sorry bastard. You blocked me f...,1,0,1,0,1,0
3,5ac105368a34b102,Action Game \nThere was also an Action Game ve...,0,0,0,0,0,0
4,f4fc7058b50a44e8,"""\n\n """"Right to be forgotten"""" \n\nI think th...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
31291,3b74648e9983d6a7,One more \n\nTesting 1-2.,0,0,0,0,0,0
31292,ee8ae4d4a8a4995e,I was just working on a post for this on your ...,0,0,0,0,0,0
31293,bfb03a234fe0f82d,I know you block mine . Have enough fun? You ...,0,0,0,0,0,0
31294,78af0060320dafd3,regarding vandalism: he culprit was caught and...,0,0,0,0,0,0


In [ ]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,3f252c5b8db86046,Christmas Message \n\nI hope you die in hell y...,1,1,1,1,1,0
1,eb472c7abd1cc98d,subversive activities\nJoseph Cardinal support...,0,0,0,0,0,0
2,774bfac348db66c6,{{unblock|You sorry bastard. You blocked me f...,1,0,1,0,1,0
3,5ac105368a34b102,Action Game \nThere was also an Action Game ve...,0,0,0,0,0,0
4,f4fc7058b50a44e8,"""\n\n """"Right to be forgotten"""" \n\nI think th...",0,0,0,0,0,0


In [ ]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,3f252c5b8db86046,Christmas Message \n\nI hope you die in hell y...,1,1,1,1,1,0
1,eb472c7abd1cc98d,subversive activities\nJoseph Cardinal support...,0,0,0,0,0,0
2,774bfac348db66c6,{{unblock|You sorry bastard. You blocked me f...,1,0,1,0,1,0
3,5ac105368a34b102,Action Game \nThere was also an Action Game ve...,0,0,0,0,0,0
4,f4fc7058b50a44e8,"""\n\n """"Right to be forgotten"""" \n\nI think th...",0,0,0,0,0,0


In [ ]:
data['toxic'].sum()

15296

In [ ]:
data=data.drop(['id'],axis=1)

In [ ]:
# data = data.sample(n=50000, random_state=42)

In [ ]:
x=data['comment_text']
y=data[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]

In [ ]:
data.shape

(31296, 7)

In [ ]:
x=x.tolist()

In [ ]:
import re
def rem(s):
  s=s.replace('\n',"")
  s=re.sub(r'[^A-Za-z0-9\s ]', '', s)

  return s.lower()

In [ ]:

x=list(map(rem,x))

In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
x = [sentence.split() for sentence in x]

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def filter(sentence):
  return [word for word in sentence if word not in stop_words]

x=list(map(filter,x))




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
word2vec_model = Word2Vec(sentences=x, vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
def get_sentence_embedding(sentence):
    return [word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]

embeddings = [get_sentence_embedding(sentence) for sentence in x]


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

embeddings = pad_sequences(embeddings, maxlen=200, dtype='float32', padding='post')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding


In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, random_state=42)

In [ ]:
embeddings=np.array(embeddings)

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(embeddings.shape[1], embeddings.shape[2])))
model.add(Dense(6, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 104s 161ms/step - accuracy: 0.9839 - loss: 0.3991 - val_accuracy: 0.9970 - val_loss: 0.2806
Epoch 2/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 100s 160ms/step - accuracy: 0.9919 - loss: 0.2588 - val_accuracy: 0.9920 - val_loss: 0.2054
Epoch 3/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 143s 161ms/step - accuracy: 0.9944 - loss: 0.2023 - val_accuracy: 0.9954 - val_loss: 0.1892
Epoch 4/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 115s 184ms/step - accuracy: 0.9956 - loss: 0.1947 - val_accuracy: 0.9966 - val_loss: 0.1847
Epoch 5/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 96s 154ms/step - accuracy: 0.9959 - loss: 0.1888 - val_accuracy: 0.9968 - val_loss: 0.1860
Epoch 6/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 145s 159ms/step - accuracy: 0.9957 - loss: 0.1816 - val_accuracy: 0.9968 - val_loss: 0.1833
Epoch 7/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 141s 158ms/step - accuracy: 0.9965 - loss: 0.1810 - val_accuracy: 0.9970 - val_loss: 0.1827
Epoch 8/10
626/626 ━━━━━━━━━━━━━━━━━━━━ 108s 173ms/step - accuracy: 0.9970 - 

In [ ]:
X_test

array([[[ 1.20473973e-01, -3.42539966e-01,  4.16474164e-01, ...,
          2.01609328e-01, -1.09734797e+00,  4.19232100e-01],
        [ 2.04008818e-01, -4.18652654e-01,  1.22317165e-01, ...,
         -1.94170505e-01, -8.76789749e-01,  3.46216410e-01],
        [ 5.92761695e-01, -6.02688253e-01,  4.00423780e-02, ...,
         -3.69734287e-01, -1.56705189e+00,  1.91350117e-01],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 3.30383293e-02, -2.05616131e-01, -2.10701171e-02, ...,
         -7.90791437e-02, -1.50678635e-01,  3.16903703e-02],
        [ 4.14586008e-01, -9.95900214e-01,  2.62887273e-02, ...,
         -6.60151482e-01, -8.37870613e

In [ ]:
y_test

,toxic,severe_toxic,obscene,threat,insult,identity_hate
25656,1,0,1,0,1,0
5405,0,0,0,0,0,0
4034,1,0,0,0,1,0
17935,0,0,0,0,0,0
21549,0,0,0,0,0,0
...,...,...,...,...,...,...
6531,0,0,0,0,0,0
26775,0,0,0,0,0,0
1397,0,0,0,0,0,0
5312,1,1,1,0,1,0


In [ ]:
model.save('toxic.h5')

In [ ]:
pred=model.predict(X_test)
predictions=(pred > 0.5).astype(int)

196/196 ━━━━━━━━━━━━━━━━━━━━ 11s 56ms/step


In [ ]:
predictions

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences


def rem(s):
    s = s.replace('\n', "")
    s = re.sub(r'[^A-Za-z0-9\s]', '', s)
    return s.lower()


test_sentence = "you as hle"


cleaned_sentence = rem(test_sentence)


from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def filter(sentence):
    return [word for word in sentence.split() if word not in stop_words]

filtered_sentence = filter(cleaned_sentence)


def get_sentence_embedding(sentence):
    return [word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]

sentence_embedding = get_sentence_embedding(filtered_sentence)


MAX_LENGTH = 200
sentence_embedding_padded = pad_sequences([sentence_embedding], maxlen=MAX_LENGTH, padding='post', dtype='float32')


predicted_probabilities = model.predict(sentence_embedding_padded)


predicted_labels = (predicted_probabilities > 0.5).astype(int)


print("Predicted Labels:", predicted_labels)


print("Predicted Probabilities:", predicted_probabilities)
